In [55]:
import pandas as pd
import os
import json
import csv
import re

import sklearn
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.linear_model import LogisticRegression

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk import word_tokenize

from chemdataextractor import Document

import numpy as np
import matplotlib.pyplot as plt

In [2]:
doi_index = []
text = []

In [3]:
pathacs = '/Users/juliachotoo/ScrapyArticles/scrapedjson/acs2/'

for filename in os.listdir(pathacs):
    with open(pathacs + filename, 'r'):
        resultacs = json.load(open(pathacs + filename, mode='r'))
        doi_index.append(resultacs['doi'])
        text.append(resultacs['text'])

In [4]:
pathspr = '/Users/juliachotoo/ScrapyArticles/scrapedjson/spr2/'

for filename in os.listdir(pathspr):
    with open(pathspr + filename, 'r'):
        resultspr = json.load(open(pathspr + filename, mode='r'))
        doi_index.append(resultspr['doi'])
        text.append(resultspr['text'])

In [5]:
pathlabels = '/Users/juliachotoo/ScrapyArticles/datalabels.csv'
labels = []

with open(pathlabels, 'r') as csvfile:
    resultlabels = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in resultlabels:
        labels.append(', '.join(row))

labels[0] = labels[0].replace('\ufeff', '')
print(labels)    


['0', '1', '0', '2', '1', '0', '0', '2', '2', '0', '2', '2', '0', '0', '0', '0', '0', '0', '1', '0', '0', '2', '0', '0', '0', '0', '0', '0', '2', '0', '0', '0', '2', '2', '0', '0', '0', '2', '0', '2', '0', '0', '0', '0', '0', '0', '1', '1', '0', '2', '0', '0', '0', '2', '2', '2', '1', '0', '0', '1', '0', '1', '2', '0', '2', '0', '2', '0', '0', '1', '0', '1', '0', '0', '2', '0', '0', '1', '0', '2', '1', '0', '0', '0', '0', '2', '2', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '2', '1', '0', '0', '0', '0', '0', '0', '0', '0', '2', '0', '0', '2', '0', '1', '0', '0', '0', '2', '2', '0', '0', '0', '2', '0', '1', '0', '0', '0', '0', '0', '2', '0', '0', '0', '0', '2', '2', '0', '2', '1', '1', '2', '1', '2', '1', '2', '0', '0', '0', '0', '0', '0', '0', '2', '0', '2', '0', '0', '2', '2', '0', '0', '0', '0', '0', '2', '2', '1', '2', '0', '2', '1', '1', '1', '1', '0', '2', '1', '0', '0', '2', '0', '0', '0', '2', '0', '0', '0', '0',

In [6]:
labels.count('2')

68

In [7]:
df = pd.DataFrame({'data': text, 'labels': labels})
print(df.loc[0:9])

                                                data labels
0  10.1021/ic501413r Metal–organic frameworks (MO...      0
1  10.1021/jp102881e  Top of PageAbstractIntroduc...      1
2  10.1021/acs.analchem.5b00391 Metal–organic fra...      0
3  10.1021/ct400255c Top of PageAbstractIntroduct...      2
4  10.1021/ie071645b  Top of PageAbstract1. Intro...      1
5  10.1021/ic502478u Metal–organic frameworks (MO...      0
6  10.1021/cm4034319 Top of PageAbstractIntroduct...      0
7  10.1021/acs.langmuir.5b04185 In the original a...      2
8  10.1021/ic060568u  Top of PageAbstractIntroduc...      2
9  10.1021/ie500310c Top of PageAbstractIntroduct...      0


In [8]:
X = df.data
y = df[df.columns[1]]
print(X.shape)
print(y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(313,)
(313,)
(234,)
(79,)
(234,)
(79,)


In [9]:
def stemming_tokenizer(text):
    stemmer = PorterStemmer()
    return [stemmer.stem(w) for w in word_tokenize(text)]
vect = CountVectorizer(tokenizer=stemming_tokenizer, stop_words=stopwords.words('english'))
vect.fit(X_train)
X_train_dtm = vect.fit_transform(X_train)


X_test_dtm = vect.transform(X_test)
X_test_dtm

<79x44773 sparse matrix of type '<class 'numpy.int64'>'
	with 64936 stored elements in Compressed Sparse Row format>

In [28]:
#tf_transformer = TfidfTransformer()
#X_train_tf = tf_transformer.fit_transform(X_train_dtm)
#X_test_tf = tf_transformer.transform(X_test_dtm)

In [10]:
nb = MultinomialNB()

In [11]:
%time nb.fit(X_train_dtm, y_train)

CPU times: user 7.56 ms, sys: 2.78 ms, total: 10.3 ms
Wall time: 8.49 ms


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [12]:
y_pred_class = nb.predict(X_test_dtm)

In [13]:
metrics.accuracy_score(y_test, y_pred_class)

0.759493670886076

In [14]:
metrics.confusion_matrix(y_test, y_pred_class)

array([[52,  0,  0],
       [ 8,  2,  0],
       [ 8,  3,  6]])

In [15]:
logreg = LogisticRegression()

In [16]:
%time logreg.fit(X_train_dtm, y_train)

CPU times: user 292 ms, sys: 3.58 ms, total: 295 ms
Wall time: 294 ms


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [17]:
y_pred_class2 = logreg.predict(X_test_dtm)

In [18]:
metrics.accuracy_score(y_test, y_pred_class2)

0.78481012658227844

In [20]:
metrics.confusion_matrix(y_test, y_pred_class2)

array([[49,  2,  1],
       [ 5,  4,  1],
       [ 5,  3,  9]])

In [32]:
from sklearn.svm import SVC


In [38]:
svcm = SVC(kernel='linear')
%time svcm.fit(X_train_dtm, y_train)

CPU times: user 520 ms, sys: 2.55 ms, total: 523 ms
Wall time: 522 ms


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [39]:
y_pred_class3 = svcm.predict(X_test_dtm)

In [40]:
metrics.accuracy_score(y_test, y_pred_class3)

0.83544303797468356

In [41]:
metrics.confusion_matrix(y_test, y_pred_class3)

array([[51,  0,  1],
       [ 6,  4,  0],
       [ 4,  2, 11]])

In [61]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

In [110]:
clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), algorithm='SAMME', n_estimators=25)
clf.fit(X_train_dtm, y_train)

AdaBoostClassifier(algorithm='SAMME',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          learning_rate=1.0, n_estimators=25, random_state=None)

In [111]:
y_pred_class4 = clf.predict(X_test_dtm)

In [112]:
metrics.accuracy_score(y_test, y_pred_class4)

0.88607594936708856

In [113]:
metrics.confusion_matrix(y_test, y_pred_class4)

array([[49,  3,  0],
       [ 2,  7,  1],
       [ 3,  0, 14]])